In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install websockets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 990.9 kB/s eta 0:00:00


In [ ]:
path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/ac/utils/enviados_has_filtros.txt"
path_drive_input2 = f"/content/drive/MyDrive/DataBase/world_bank/ac/Despesas/despesas"


# Verificar IP

In [ ]:
import requests

my_country = requests.get("https://api.myip.com/")
my_country.json()

{'ip': '34.42.43.17', 'country': 'United States', 'cc': 'US'}

# Importação

In [ ]:
import requests
import ast
import asyncio
import websockets
import nest_asyncio
import json
from time import sleep
import os
import shutil
from zipfile import ZipFile
import pandas as pd


def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)


def send_solo_drive() -> None:

  drive.mount('/content/drive')

  send_folder_drive(path_drive_input2 + "_auto",
                    "/content/input/")

  # send_folder_drive(path_drive_csv + "_auto",
  #                   "/content/csvs")


def ler(path: str) -> str:

  with open(path, "r", encoding="utf-8") as txt:
    return txt.read()



def extrair_zip(path, path_output) -> None:
  # loading the temp.zip and creating a zip object
  with ZipFile(path, 'r') as zObject:

      # Extracting all the members of the zip
      # into a specific location.
      zObject.extractall(
          path=path_output)


def save_csv(path: str, dados: list) -> None:

  columns = ["empenho", "ano", "orgao",
             "credor", "empenhada", "liquidada",
             "paga", "descricao", "data"]
  df = pd.DataFrame(dados, columns=columns)
  df.to_csv(path, index=False)


def create_table(dados: dict) -> list:

  try:

    number_columns = len(dados["result"]['qDataPages'][0]['value'])
    number_rows = len(dados["result"]['qDataPages'][0]['value'][2]['qMatrix'])

    columns_values = []
    for n_column in range(number_columns):
      atalho = dados["result"]['qDataPages'][0]['value'][n_column]['qMatrix']
      columns_values.append([atalho[n_row][0]["qText"] for n_row in range(number_rows) ])
    table = list(zip(*columns_values))

    return table

  except:
    return []


def make_envios_get_data(ano_number: int, municipio_number: int) -> list:

  # path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/ac/utils/coleta_direta_envios.txt"
  path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/ac/utils/enviados_has_filtros.txt"

  has = ler(path_drive_input)
  has = json.loads(has)

  for xn in has["log"]["entries"]:
    ws_messages = xn.get("_webSocketMessages")
    if ws_messages:
      break

  sends_hs = [mensage["data"] for mensage in ws_messages if mensage["type"] == "send"]
  sends_hs = [send for send in sends_hs if not "GetLayout" in send and not "GetEffectiveProperties" in send]
  consulta = '{"delta": true, "handle": 8, "method": "GetHyperCubeData", "params": ["/qHyperCubeDef", [{"qTop": 0, "qLeft": 0, "qHeight": 50, "qWidth": 1}, {"qTop": 0, "qLeft": 1, "qHeight": 50, "qWidth": 1}, {"qTop": 0, "qLeft": 2, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 3, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 4, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 5, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 6, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 7, "qHeight": 50, "qWidth": 1}, {"qTop": 0, "qLeft": 8, "qHeight": 50, "qWidth": 1}]], "id": 248, "jsonrpc": "2.0"}'.replace("50", "500")
  sends_hs = sends_hs[:62]

  get_data = [[n_send, send] for n_send, send in enumerate(sends_hs) if "GetHyperCubeData" in send]

  remove_data = [row[0] for row in get_data[:-1]]

  for remove in sorted(remove_data, reverse=True):
    del sends_hs[remove]

  # sends_hs[47] = sends_hs[47].replace('"/qListObjectDef",[8]', f'"/qListObjectDef",[{ano_number}]')
  # sends_hs[58] = sends_hs[58].replace('"/qListObjectDef",[0]', f'"/qListObjectDef",[{municipio_number}]')

  sends_hs[45] = sends_hs[45].replace('"/qListObjectDef",[9]', f'"/qListObjectDef",[{ano_number}]')
  sends_hs[55] = sends_hs[55].replace('"/qListObjectDef",[0]', f'"/qListObjectDef",[{municipio_number}]')

  sends_hs[-1] = consulta

  return sends_hs

# Extrair CSV

In [ ]:
# extrair_zip(path=path_drive_csv + ".zip",
#         path_output="/content/csvs")

In [ ]:
extrair_zip(path=path_drive_input + ".zip",
        path_output="/content/input")

### Extrair Auto-Save

In [ ]:
# extrair_zip(path=path_drive_csv2 + "_auto" + '.zip',
#         path_output="/content/csvs")

In [ ]:
extrair_zip(path=path_drive_input2 + "_auto" + '.zip',
        path_output="/content/input")

# Carregar Envios

In [ ]:
path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/ac/utils/enviados_has_filtros.txt"
has = ler(path_drive_input)
has = json.loads(has)

In [ ]:
for xn in has["log"]["entries"]:
  ws_messages = xn.get("_webSocketMessages")
  if ws_messages:
    break
sends_hs = [mensage["data"] for mensage in ws_messages if mensage["type"] == "send"]
sends_hs = [send for send in sends_hs if not "GetLayout" in send and not "GetEffectiveProperties" in send]

In [ ]:
porta = '{"delta": true, "handle": 8, "method": "GetHyperCubeData", "params": ["/qHyperCubeDef", [{"qTop": 0, "qLeft": 0, "qHeight": 50, "qWidth": 1}, {"qTop": 0, "qLeft": 1, "qHeight": 50, "qWidth": 1}, {"qTop": 0, "qLeft": 2, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 3, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 4, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 5, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 6, "qHeight": 50, "qWidth": 1},  {"qTop": 0, "qLeft": 7, "qHeight": 50, "qWidth": 1}, {"qTop": 0, "qLeft": 8, "qHeight": 50, "qWidth": 1}]], "id": 248, "jsonrpc": "2.0"}'.replace("50", "1000")

In [ ]:
get_data = [[n_send, send] for n_send, send in enumerate(sends_hs) if "GetHyperCubeData" in send]

In [ ]:
get_data

In [ ]:
select_filtes = [[n_send, send] for n_send, send in enumerate(sends_hs) if "SelectListObjectValues" in send]

In [ ]:
select_filtes

In [ ]:
sends_hs = sends_hs[:62]

In [ ]:
remove_data = [row[0] for row in get_data[:-1]]
remove_data

[35, 39, 45, 50]

In [ ]:
for remove in sorted(remove_data, reverse=True):
  del sends_hs[remove]

In [ ]:
ano_number = 8
municipio_number = 0
sends_hs[47] = sends_hs[47].replace('"/qListObjectDef",[8]', f'"/qListObjectDef",[{ano_number}]')
sends_hs[58] = sends_hs[58].replace('"/qListObjectDef",[8]', f'"/qListObjectDef",[{municipio_number}]')
sends_hs[-1] = porta

# Pegar anos e Municipios

In [ ]:
path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/ac/utils/enviados_has_filtros.txt"
has = ler(path_drive_input)
has = json.loads(has)
for xn in has["log"]["entries"]:
  ws_messages = xn.get("_webSocketMessages")
  if ws_messages:
    break
sends_hs = [mensage["data"] for mensage in ws_messages if mensage["type"] == "send"]
sends_hs = [send for send in sends_hs if not "GetLayout" in send and not "GetEffectiveProperties" in send]

In [ ]:
sends_hs = sends_hs[:57]

In [ ]:
get_list = [[n_send, send] for n_send, send in enumerate(sends_hs) if "GetListObjectData" in send]

In [ ]:
remove_data = [row[0] for row in get_list[1:-1]]
remove_data

[44, 47, 52]

In [ ]:
for remove in sorted(remove_data, reverse=True):
  del sends_hs[remove]

## Coleta de anos e municipio

In [ ]:
async def scrape_website_ano_municipio():
    # Replace the URL with the WebSocket endpoint of the target website

    websocket_url = "ws://qlik.tceac.tc.br/app/cb9dbe12-5f79-4af9-93a6-a778ae7ffffa?reloadUri=http%3A%2F%2Fqlik.tceac.tc.br%2Fextensions%2Fgastospublicos%2Fgastospublicos.html%3F_ga%3D2.234985153.1351290455.1696941468-160473323.1696941468%26_gl%3D1*1udkbll*_ga*MTYwNDczMzIzLjE2OTY5NDE0Njg.*_ga_7W9X95Q11R*MTY5Njk0MTQ2Ny4xLjEuMTY5Njk0MTU4NS4wLjAuMA..*_ga_BGWR95NM02*MTY5Njk0MTQ2Ny4xLjEuMTY5Njk0MTU4NS4wLjAuMA.."

    dados = []
    dado = None

    async with websockets.connect(websocket_url) as websocket:
        print(f"Connected to {websocket_url}")

        for x in range(2):
          message = await websocket.recv()
          print(f"Received message: {message}")

        for send in sends_hs:

          await websocket.send(send)

          message = await websocket.recv()

          if "GetListObjectData" in send:
            print(f"Received message: {message}")
            dado = json.loads(message)
            dados.append(dado)

    return dados

# Run the scraper
if __name__ == "__main__":
  nest_asyncio.apply()
  dados = asyncio.get_event_loop().run_until_complete(asyncio.gather(scrape_website_ano_municipio()))[0]


Connected to ws://qlik.tceac.tc.br/app/cb9dbe12-5f79-4af9-93a6-a778ae7ffffa?reloadUri=http%3A%2F%2Fqlik.tceac.tc.br%2Fextensions%2Fgastospublicos%2Fgastospublicos.html%3F_ga%3D2.234985153.1351290455.1696941468-160473323.1696941468%26_gl%3D1*1udkbll*_ga*MTYwNDczMzIzLjE2OTY5NDE0Njg.*_ga_7W9X95Q11R*MTY5Njk0MTQ2Ny4xLjEuMTY5Njk0MTU4NS4wLjAuMA..*_ga_BGWR95NM02*MTY5Njk0MTQ2Ny4xLjEuMTY5Njk0MTU4NS4wLjAuMA..
Received message: {"jsonrpc":"2.0","method":"OnAuthenticationInformation","params":{"userId":"anonymousd198541e-78e6-47a2-b5bc-92065b96b613","userDirectory":"NONE","serverNodeId":"7df7e972-1b99-4be9-b360-1c72db7a14fd","mustAuthenticate":false}}
Received message: {"jsonrpc":"2.0","method":"OnConnected","params":{"qSessionState":"SESSION_CREATED"}}
Received message: {"jsonrpc":"2.0","id":84,"delta":true,"result":{"qDataPages":[{"op":"add","path":"/","value":[{"qMatrix":[[{"qText":"2023","qNum":2023,"qElemNumber":9,"qState":"S"}],[{"qText":"2014","qNum":2014,"qElemNumber":0,"qState":"A"}],[{"qT

In [ ]:
def get_anos_municipios_from_dados(dados: list) -> list:

  infos = []
  for dado in dados:
    path_temp = dado["result"]["qDataPages"][0]["value"][0]["qMatrix"]
    info = {path[0]["qText"]: path[0]["qElemNumber"] for path in path_temp}
    infos.append(info)

  return infos

In [ ]:
anos, municipios = get_anos_municipios_from_dados(dados)

# Pegar dados diretos

In [ ]:
async def scrape_website():
    # Replace the URL with the WebSocket endpoint of the target website

    websocket_url = "ws://qlik.tceac.tc.br/app/cb9dbe12-5f79-4af9-93a6-a778ae7ffffa?reloadUri=http%3A%2F%2Fqlik.tceac.tc.br%2Fextensions%2Fgastospublicos%2Fgastospublicos.html%3F_ga%3D2.234985153.1351290455.1696941468-160473323.1696941468%26_gl%3D1*1udkbll*_ga*MTYwNDczMzIzLjE2OTY5NDE0Njg.*_ga_7W9X95Q11R*MTY5Njk0MTQ2Ny4xLjEuMTY5Njk0MTU4NS4wLjAuMA..*_ga_BGWR95NM02*MTY5Njk0MTQ2Ny4xLjEuMTY5Njk0MTU4NS4wLjAuMA.."

    dados = []
    pagina = 0
    dado = None

    async with websockets.connect(websocket_url, ping_interval=None) as websocket:
        # print(f"Connected to {websocket_url}")

        for x in range(2):
          message = await websocket.recv()
          # print(f"Received message: {message}")

        for send in sends_hs:

          await websocket.send(send)

          message = await websocket.recv()

          if "GetHyperCubeData" in send:
            # print(f"Received message: {message}")
            dado = json.loads(message)
            dado = create_table(dado)
            dados += dado
            break

        while dado:

          # print("Proxima Pagina")
          send = send.replace(f'"qTop": {pagina},', f'"qTop": {pagina + 500},')
          await websocket.send(send)
          message = await websocket.recv()
          dado = json.loads(message)
          dado = create_table(dado)
          dados += dado
          pagina += 500
          # print("Intervalo")
          sleep(1)


    return dados

# Run the scraper
if __name__ == "__main__":

  nest_asyncio.apply()

  for text_municipio, municipio_number, in municipios.items():
    for text_ano, ano_number, in anos.items():

      print(f"Coletando {text_municipio} ano: {text_ano}")

      path_base = f"/content/input/{text_ano}"
      path = path_base + f"/{text_municipio}.csv"

      if not os.path.exists(path):

        sends_hs = make_envios_get_data(ano_number=ano_number, municipio_number=municipio_number)
        dados = asyncio.get_event_loop().run_until_complete(asyncio.gather(scrape_website()))
        dados = list(set(dados[0]))

        os.makedirs(path_base, exist_ok=True)

        save_csv(path, dados)

        print("Intervalo da sessão")
        sleep(5)
        print("Mandando drive")
        send_solo_drive()
        print("-" * 30)

Coletando ACRELÂNDIA ano: 2023
Coletando ACRELÂNDIA ano: 2014
Coletando ACRELÂNDIA ano: 2015
Coletando ACRELÂNDIA ano: 2016
Coletando ACRELÂNDIA ano: 2017
Coletando ACRELÂNDIA ano: 2018
Coletando ACRELÂNDIA ano: 2019
Coletando ACRELÂNDIA ano: 2020
Coletando ACRELÂNDIA ano: 2021
Coletando ACRELÂNDIA ano: 2022
Coletando ASSIS BRASIL ano: 2023
Coletando ASSIS BRASIL ano: 2014
Coletando ASSIS BRASIL ano: 2015
Coletando ASSIS BRASIL ano: 2016
Coletando ASSIS BRASIL ano: 2017
Coletando ASSIS BRASIL ano: 2018
Coletando ASSIS BRASIL ano: 2019
Coletando ASSIS BRASIL ano: 2020
Coletando ASSIS BRASIL ano: 2021
Coletando ASSIS BRASIL ano: 2022
Coletando BRASILÉIA ano: 2023
Coletando BRASILÉIA ano: 2014
Coletando BRASILÉIA ano: 2015
Coletando BRASILÉIA ano: 2016
Coletando BRASILÉIA ano: 2017
Coletando BRASILÉIA ano: 2018
Coletando BRASILÉIA ano: 2019
Coletando BRASILÉIA ano: 2020
Coletando BRASILÉIA ano: 2021
Coletando BRASILÉIA ano: 2022
Coletando BUJARI ano: 2023
Coletando BUJARI ano: 2014
Coleta

# Mandar drive

In [ ]:
drive.mount('/content/drive')


send_folder_drive(path_drive_input2,
                "/content/input/")

# send_folder_drive(path_drive_csv,
#                 "/content/csvs")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
